# For the selected animal, plot peri-stimulus activity of all its neurons and save the figure

In [7]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import pandas as pd
import math

from tms_tg import TMSTG, EPOCHISOLATORS

# Setting the number of threads for Numba
nb.set_num_threads(nb.config.NUMBA_NUM_THREADS)

# file containing the path to data location of a particular animal
animalList = r'G:\Vishnu\data\animalList\animalList.xlsx'

# Instantiate a TMSTG object using this file
tms = TMSTG.load(animalList)


G:\Vishnu\Analysis\tms-tg\lib\helper_tms_tg.py:245: UserWarning: Cannot check the MSO order in blocksinfo for epoch ('20210307', 'MC', 'L5', 'none', 'none', '1409') using mat-data information. Hence, using the MSO order in blocksinfo as it is; keep in mind that the source of this MSO order contained in infofile has to match the true recording MSO order, otherwise wrong MSO will be associated to epochs [Katastrophe].
  warnings.warn(f'Cannot check the MSO order in blocksinfo for epoch {epochIndex} using mat-data information.'
G:\Vishnu\Analysis\tms-tg\lib\helper_tms_tg.py:245: UserWarning: Cannot check the MSO order in blocksinfo for epoch ('20210307', 'MC', 'L5', 'none', 'none', '1410') using mat-data information. Hence, using the MSO order in blocksinfo as it is; keep in mind that the source of this MSO order contained in infofile has to match the true recording MSO order, otherwise wrong MSO will be associated to epochs [Katastrophe].
  warnings.warn(f'Cannot check the MSO order in b

In [8]:
# Get the directory name containing data for this particular animal
dirname = pd.read_excel(animalList).dropna().loc[0, 'Folder']

# Get unique blocks from blocks information
blocks = tms.blocksinfo.index.unique()

# Iterate over each block
for block in blocks:
    # Replace 'none' with None in the block list
    block = [None if item == 'none' else item for item in block]
    
    # Set analysis parameters for selecting all blocks
    tms.analysis_params = {'selectionParams': {'Epoch': dict(zip(EPOCHISOLATORS, block)), }}
    
    # Calculate average firing rate activity around the stimulus
    meanPSFR, t, meanBaselineFR, _ = tms.avg_FR_per_neuron(squeezeDim=False)
    
    # Get maximum firing rate for use in y-axis limits
    maxFR = np.max(meanPSFR)
    
    # Filter blocks and get selected blocks and their indices
    selectBlocks, selectBlocksIdx = tms.filter_blocks
    
    # Define the layout (number of rows and columns) for subplots
    numRows = 12
    numCols = math.ceil(len(meanPSFR) / numRows)
    
    # Create a figure and subplots
    fig, axes = plt.subplots(min(numRows, len(meanPSFR)), numCols, figsize=(12, 10))
    fig.suptitle(block)  # Set title for the figure
    
    # Iterate over each neuron
    for index in range(len(meanPSFR)):
        row = index % numRows
        col = index // numRows
        if len(meanPSFR) == 1 and numCols == 1:
            ax = axes
        elif numCols == 1:
            ax = axes[row]
        else:
            ax = axes[row][col]
        
        # Plot mean PSFR for the neuron
        ax.plot(t, meanPSFR[index].squeeze())
        ax.set_ylabel('MT-\n' + str(selectBlocks.iloc[index, 1]), fontsize=10)  # Set y-axis label
        ax.set_ylim(0, maxFR)  # Set y-axis limits
        
        # Set title for the subplot based on block information
        if any(boolIndex := pd.Series(selectBlocks.columns).str.match('Skin-Injection')):
            ax.set_title(selectBlocks.iloc[index, :].loc[boolIndex.values].item())
        
        # Add legend for the subplot
        if index == 0:
            ax.legend(['Neuron - ' + str(item + 1) for item in range(meanPSFR[index].shape[2])])
    
    # Save the figure
    plt.savefig(dirname + '\\Figures\\' + str(block) + '.jpg', dpi='figure', format='jpeg')
    plt.close()  # Close the figure to release memory



analysis_params set to:  {'selectionParams': {'Epoch': {'Animal': ('20210307',), 'Region': ('MC',), 'Layer': ('L5',), 'CoilHemVsRecHem': None, 'Mov': None, 'Depth': ('1409',)}}, 'TMSArtifactParams': {'timeWin': (-0.3, 0.5)}, 'peristimParams': {'smoothingParams': {'win': 'gaussian', 'width': 1.5, 'overlap': 0.3333333333333333}, 'timeWin': (-50.0, 100.0), 'trigger': 'TMS', 'baselinetimeWin': (-50.0, -1.0)}}
----------------------------
compute_firingrate runs...........
compute_firingrate runs...........
analysis_params set to:  {'selectionParams': {'Epoch': {'Animal': ('20210307',), 'Region': ('MC',), 'Layer': ('L5',), 'CoilHemVsRecHem': None, 'Mov': None, 'Depth': ('1410',)}}, 'TMSArtifactParams': {'timeWin': (-0.3, 0.5)}, 'peristimParams': {'smoothingParams': {'win': 'gaussian', 'width': 1.5, 'overlap': 0.3333333333333333}, 'timeWin': (-50.0, 100.0), 'trigger': 'TMS', 'baselinetimeWin': (-50.0, -1.0)}}
----------------------------
compute_firingrate runs...........
compute_firingrate 

In [28]:
tms.blocksinfo

MSO    MT  no. of Trigs  \
Animal   Region Layer CoilHemVsRecHem Mov  Depth                            
20170212 MC     L23   none            none 454      64  0.8            50   
                                           454      69  0.9           100   
                                           454       0  0.0            50   
                                           454      76  1.0           100   
                                           454      83  1.1           100   
                                           454      92  1.2           138   
                L4    none            none 752      71  0.9            50   
                                           752      77  1.0           100   
                                           752      82  1.1            50   
                                           752      92  1.2           100   
                L5    none            none 1214     64  0.8            50   
                                           1214     69  0.9           100   
                                           1214      0  0.0            50   
                                           1214     76  1.0           149   
                                           1214     85  1.1            46   
                                           1215     71  0.9            50   
                                           1215     77  1.0           100   
                                           1215     82  1.1           100   
                                           1215     91  1.2           150   

                                                  Stimpulses StimHem CoilDir  \
Animal   Region Layer CoilHemVsRecHem Mov  Depth                               
20170212 MC     L23   none            none 454            50    none      ML   
                                           454           100    none      ML   
                                           454            50    none      ML   
                                           454           100    none      ML   
                                           454            90    none      ML   
                                           454           130    none      ML   
                L4    none            none 752            50    none      ML   
                                           752           100    none      ML   
                                           752            50    none      ML   
                                           752            75    none      ML   
                L5    none            none 1214           50    none      ML   
                                           1214          100    none      ML   
                                           1214           50    none      ML   
                                           1214          145    none      ML   
                                           1214           45    none      ML   
                                           1215           50    none      ML   
                                           1215          100    none      ML   
                                           1215           66    none      ML   
                                           1215           94    none      ML   

                                                 TG-Injection  RecArea   \
Animal   Region Layer CoilHemVsRecHem Mov  Depth                          
20170212 MC     L23   none            none 454              No       MC   
                                           454              No       MC   
                                           454              No       MC   
                                           454              No       MC   
                                           454              No       MC   
                                           454              No       MC   
                L4    none            none 752              No       MC   
                                           752              No       MC   
 

In [13]:
import pandas as pd
df = pd.read_excel('G:/Vishnu/data/animalList/animalList.xlsx').dropna()

In [14]:
df

,Unnamed: 0,Animal,Folder
0,0,20191201,G:\Vishnu\data\TMSTG\20191201


In [7]:
df.loc[0,'Folder'] = 'G:\\Vishnu\\data\\TMSTG\\20191201'
df.loc[0,'Animal'] = '20191201'

In [19]:
df

,Unnamed: 0,Animal,Folder
0,0,20191201,G:\Vishnu\data\TMSTG\20191201


In [20]:
df.drop(columns='Unnamed: 0', inplace=True)

In [21]:
df.to_excel('G:/Vishnu/data/animalList/animalList.xlsx', index=False)

In [3]:
dirname

'G:\\Vishnu\\data\\TMSTG\\20191119'